<a href="https://colab.research.google.com/github/hojuna/Black-box_Optimization_dacon/blob/main/search_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.model_selection import RandomizedSearchCV


# Load the data
train_df_org = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/test.csv')

In [ ]:
df_shuffled= train_df_org
# 균등하게 나누기 위해 임의로 데이터를 8:2로 분할
split_index = int(len(df_shuffled) * 0.8)
train_df, valiation_df = df_shuffled[:split_index], df_shuffled[split_index:]


In [ ]:

def val_score(model, valiation_df):
  id_y=pd.DataFrame()

  top_5_percent_valiation_df = valiation_df.sort_values(by='y').reset_index(drop=True)[-int(0.05 * len(valiation_df)):]
  #id만 분리 valiation_df, y값 제거
  X = valiation_df.iloc[:, 1:-1]
  id_y['ID']=valiation_df.iloc[:,0]

  #valiation_df y값 예측
  y_pred=model.predict(X)
  id_y['y_pred']=y_pred

  # print(id_x.head)
  # #id 복원

  #상위 10% valiation_df 구하기
  top_10_percent_pred_valiation=id_y.sort_values(by='y_pred').reset_index(drop=True)[-int(0.10 * len(id_y)):]

  # ID 값에서 공백 제거
  top_5_percent_valiation_df['ID'] = top_5_percent_valiation_df['ID'].str.strip()
  top_10_percent_pred_valiation['ID'] = top_10_percent_pred_valiation['ID'].str.strip()

  #top_5_percent_valiation_df 중  상위 10%에 포함되는 비율을 구해서 반환
  return top_5_percent_valiation_df['ID'].isin(top_10_percent_pred_valiation['ID']).sum()/len(top_5_percent_valiation_df)

In [ ]:

X = train_df.iloc[:, 1:-1]  # Features
y = train_df['y']  # Target

In [ ]:
# #RandomForestRegressor
# # Train model
# RandomForestRegressor_model = RandomForestRegressor(n_estimators=300, random_state=42)
# RandomForestRegressor_model.fit(X, y)
# score=val_score(RandomForestRegressor_model,valiation_df)
# print("RandomForestRegressor_model:" , score)

In [ ]:
from sklearn.linear_model import LinearRegression

# 모델 초기화
model = LinearRegression()

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("LinearRegression:" ,score)
# LinearRegression: 0.8753117206982544

LinearRegression: 0.8753117206982544


In [ ]:
from sklearn.linear_model import Ridge

# 모델 초기화
model = Ridge(alpha=1.0)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("Ridge:" , score)


Ridge: 0.8753117206982544


In [ ]:
from sklearn.linear_model import Lasso

# 모델 초기화
model = Lasso(alpha=0.1)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("Lasso:" , score)


Lasso: 0.8528678304239401


In [ ]:
from sklearn.linear_model import ElasticNet

# 모델 초기화
model = ElasticNet(alpha=0.1, l1_ratio=0.5)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("ElasticNet:" , score)


ElasticNet: 0.8528678304239401


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# 모델 초기화
model = DecisionTreeRegressor()

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("DecisionTreeRegressor:" , score)


DecisionTreeRegressor: 0.7780548628428927


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 모델 초기화
model = RandomForestRegressor(n_estimators=100)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("RandomForestRegressor_model:" , score)


RandomForestRegressor_model: 0.8703241895261845


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# 모델 초기화
model = GradientBoostingRegressor(n_estimators=100)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("GradientBoostingRegressor:" , score)


GradientBoostingRegressor: 0.8728179551122195


In [ ]:
from xgboost import XGBRegressor

# 모델 초기화
model = XGBRegressor(n_estimators=100)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("XGBRegressor:" , score)


XGBRegressor: 0.8628428927680798


In [ ]:
from lightgbm import LGBMRegressor

# 모델 초기화
model = LGBMRegressor(n_estimators=100)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("LGBMRegressor:" , score)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001575 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 32094, number of used features: 11
[LightGBM] [Info] Start training from score 83.626803
LGBMRegressor: 0.8728179551122195


In [ ]:
# from catboost import CatBoostRegressor

# # 모델 초기화
# model = CatBoostRegressor(iterations=100, verbose=0)

# # 모델 학습
# model.fit(X, y)

# score=val_score(model,valiation_df)
# print("CatBoostRegressor:" , score)


In [ ]:
from sklearn.svm import SVR

# 모델 초기화
model = SVR(kernel='rbf')

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("SVR:" , score)


SVR: 0.8653366583541147


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# 모델 초기화
model = KNeighborsRegressor(n_neighbors=5)

# 모델 학습
model.fit(X, y)

# 예측
score=val_score(model,valiation_df)
print("KNeighborsRegressor:" , score)



KNeighborsRegressor: 0.8503740648379052


In [ ]:
from sklearn.neural_network import MLPRegressor

# 모델 초기화
model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("MLPRegressor:" , score)


MLPRegressor: 0.8753117206982544


In [ ]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# 개별 모델 초기화
model1 = LinearRegression()
model2 = DecisionTreeRegressor()

# 앙상블 모델 초기화
model = VotingRegressor(estimators=[('lr', model1), ('dt', model2)])

# 모델 학습
model.fit(X, y)
score=val_score(model,valiation_df)
print("VotingRegressor:" , score)


VotingRegressor: 0.8678304239401496


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

# 모델 초기화
base_model = DecisionTreeRegressor()
model = BaggingRegressor(base_estimator=base_model, n_estimators=10)

# 모델 학습
model.fit(X, y)

score=val_score(model,valiation_df)
print("BaggingRegressor:" , score)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingRegressor: 0.8628428927680798


In [ ]:
###########################

In [ ]:
from sklearn.linear_model import BayesianRidge

model = BayesianRidge()
model.fit(X,y)

score=val_score(model,valiation_df)
print("BayesianRidge:" , score)

BayesianRidge: 0.8753117206982544


In [ ]:
from sklearn.linear_model import HuberRegressor

model = HuberRegressor()
model.fit(X,y)
score=val_score(model,valiation_df)
print("HuberRegressor:" , score)


HuberRegressor: 0.8778054862842892


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
# from sklearn.linear_model import QuantileRegressor

# model = QuantileRegressor(quantile=0.5)  # 중위수 회귀
# model.fit(X,y)
# score=val_score(model,valiation_df)
# print("QuantileRegressor:" , score)


In [ ]:
from sklearn.linear_model import PoissonRegressor

model = PoissonRegressor()
model.fit(X,y)
score=val_score(model,valiation_df)
print("PoissonRegressor:" , score)


PoissonRegressor: 0.8528678304239401


In [ ]:
from sklearn.linear_model import TheilSenRegressor

model = TheilSenRegressor()
model.fit(X,y)
score=val_score(model,valiation_df)
print("TheilSenRegressor:" , score)


TheilSenRegressor: 0.8778054862842892


In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit

model = OrthogonalMatchingPursuit()
model.fit(X,y)
score=val_score(model,valiation_df)
print("OrthogonalMatchingPursuit:" , score)


OrthogonalMatchingPursuit: 0.8528678304239401


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

kernel = RBF()  # RBF 커널 사용
model = GaussianProcessRegressor(kernel=kernel)
model.fit(X,y)
score=val_score(model,valiation_df)
print("GaussianProcessRegressor:" , score)


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor: 0.09725685785536159


In [ ]:
from sklearn.cross_decomposition import PLSRegression

model = PLSRegression(n_components=2)
model.fit(X,y)
score=val_score(model,valiation_df)
print("PLSRegression:" , score)


PLSRegression: 0.8753117206982544


In [ ]:
from sklearn.kernel_ridge import KernelRidge

model = KernelRidge(kernel='rbf')
model.fit(X,y)
score=val_score(model,valiation_df)
print("KernelRidge:" , score)


KernelRidge: 0.8778054862842892


In [ ]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np
from sklearn.utils import shuffle

# 데이터 샘플링
X_sample, y_sample = shuffle(X, y, random_state=42)  # 데이터 샘플링
X_sample = X_sample[:10000]  #10000개의 샘플 사용
y_sample = y_sample[:10000]

# KernelRidge 모델 초기화
model = KernelRidge()

# 하이퍼파라미터 범위 설정 (더 좁은 범위)
param_distributions = {
    'alpha': uniform(1e-2, 1e1),  # Regularization strength (범위 축소)
    'kernel': ['linear', 'rbf'],  # Kernel types (범위 축소)
    'gamma': uniform(1e-2, 1e-1),  # Kernel coefficient (범위 축소)
    'degree': [2, 3],  # Degree of the polynomial kernel (적당한 값)
    'coef0': uniform(0, 1)  # Independent term (범위 축소)
}

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=30,  # 시도할 하이퍼파라미터 조합의 수
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',  # 평가 척도
    random_state=42,  # 결과 재현을 위한 랜덤 시드
    n_jobs=-1,  # 병렬처리
    verbose=2  # 진행 상황 출력
)

# 모델 훈련 및 하이퍼파라미터 튜닝
random_search.fit(X_sample, y_sample)

# 최적의 모델 및 하이퍼파라미터 확인
best_model = random_search.best_estimator_
best_params = random_search.best_params_
print("Best KernelRidge model:", best_model)
print("Best parameters:", best_params)

# 성능 평가
score=val_score(best_model,valiation_df)
print("KernelRidge (with tuned hyperparameters):", score)


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best KernelRidge model: KernelRidge(alpha=0.06522117123602399, coef0=0.8154614284548342, degree=2,
            gamma=0.081134195274865)
Best parameters: {'alpha': 0.06522117123602399, 'coef0': 0.8154614284548342, 'degree': 2, 'gamma': 0.081134195274865, 'kernel': 'linear'}
KernelRidge (with tuned hyperparameters): 0.8778054862842892


In [ ]:


# Predict on test data
X_test = test_df.iloc[:, 1:]
y_pred = best_model.predict(X_test)

# Identify top 10% of predicted values
threshold = np.percentile(y_pred, 90)
top_10_percent_mask = y_pred >= threshold

# Create submission file
submission_df = pd.read_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/sample_submission.csv')
submission_df['y'] = y_pred
submission_df.to_csv('/content/drive/MyDrive/dacon/2024_8_data/submission/KernelRidge_submission.csv', index=False)